# Initialization

In [1]:
# import libraries
import os
import sys
import time
import pandas as pd
import numpy as np
from scipy import stats
from scipy.interpolate import CubicSpline
import torch.optim as optim
import torch
from torch.utils.data import DataLoader, TensorDataset, Subset
from scipy.stats import mode

In [2]:
num_epochs = 20
BATCH_SIZE = 128
learning_rate = 0.001
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Load Data

In [3]:
# load data without header
data1 = pd.read_csv('./ISWC21_data_plus_raw/wetlab_data.csv')
# add header
data1.columns = ['subject_id', 'acc_x', 'acc_y', 'acc_z', 'activity', 'activity_label_2']
data1.head()

,subject_id,acc_x,acc_y,acc_z,activity,activity_label_2
0,0,0.306563,9.196875,-1.22625,null_class,null_class
1,0,0.306563,9.196875,-1.22625,null_class,null_class
2,0,0.306563,9.196875,-1.22625,null_class,null_class
3,0,0.306563,9.196875,-1.22625,null_class,null_class
4,0,0.306563,9.196875,-1.22625,null_class,null_class


In [4]:
#remove activity label 2 column
data1 = data1.drop(['activity_label_2'], axis=1)
data1.shape

(3163679, 5)

In [5]:
# count number of unique subjects
print(f"number of unique subjects: {data1['subject_id'].nunique()}")

number of unique subjects: 22


In [6]:
data1['subject_id'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21], dtype=int64)

In [7]:
# load data without header
data2 = pd.read_csv('./ISWC21_data_plus_raw/rwhar_data.csv', header=None)
# add header
data2.columns = ['subject_id', 'acc_x', 'acc_y', 'acc_z', 'activity']
data2.head()

,subject_id,acc_x,acc_y,acc_z,activity
0,0,-9.57434,-2.02733,1.34506,climbing_up
1,0,-9.56479,-1.99597,1.39345,climbing_up
2,0,-9.55122,-1.98445,1.41139,climbing_up
3,0,-9.51335,-1.97557,1.42615,climbing_up
4,0,-9.52959,-1.98187,1.45395,climbing_up


In [8]:
data2.shape

(3200803, 5)

In [9]:
# print number of unique subjects
print(f"number of unique subjects: {data2['subject_id'].nunique()}")

number of unique subjects: 15


In [10]:
# load data without header
data3 = pd.read_csv('./ISWC21_data_plus_raw/sbhar_data.csv', header=None)
# add header
data3.columns = ['subject_id', 'acc_x', 'acc_y', 'acc_z', 'activity']
data3.head()

,subject_id,acc_x,acc_y,acc_z,activity
0,0,0.443056,0.037500,0.888889,null_class
1,0,0.440278,0.041667,0.880556,null_class
2,0,0.451389,0.043056,0.876389,null_class
3,0,0.456944,0.034722,0.888889,null_class
4,0,0.447222,0.036111,0.888889,null_class


In [11]:
data3.shape

(1122772, 5)

In [12]:
data3['subject_id'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], dtype=int64)

In [13]:
data3['subject_id'] = data3['subject_id'] + 22

In [14]:
data3['subject_id'].unique()

array([22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51], dtype=int64)

In [15]:
# print number of unique subjects
print(f"number of unique subjects: {data3['subject_id'].nunique()}")

number of unique subjects: 30


In [16]:
# join all data in one dataframe row-wise
# data = pd.concat([data1, data2, data3], ignore_index=True, axis=0)
data = pd.concat([data1, data3], axis=0)

In [17]:
data.shape

(4286451, 5)

In [18]:
data.sample(5)

,subject_id,acc_x,acc_y,acc_z,activity
534450,37,0.998611,0.087500,-0.175000,standing
439931,3,-5.518125,-0.306563,6.437813,null_class
1932401,13,-4.291875,-4.905000,6.131250,null_class
194902,27,-0.455556,0.636111,0.731944,null_class
1607124,10,-9.196875,0.613125,0.613125,null_class


In [19]:
# describe data
data.describe()

,subject_id,acc_x,acc_y,acc_z
count,4.286451e+06,4.286451e+06,4.286451e+06,4.286451e+06
mean,1.732692e+01,-3.142137e+00,-1.027698e+00,2.850134e+00
std,1.358579e+01,4.683064e+00,3.358477e+00,3.758104e+00
min,0.000000e+00,-3.924000e+01,-3.924000e+01,-3.924000e+01
25%,7.000000e+00,-7.050938e+00,-2.759063e+00,2.777778e-03
50%,1.400000e+01,-3.678750e+00,-2.972222e-01,2.759063e+00
75%,2.300000e+01,9.166667e-01,5.930556e-01,6.131250e+00
max,5.100000e+01,3.893344e+01,3.310875e+01,3.893344e+01


In [20]:
# check for missing values
data.isnull().sum()

subject_id    0
acc_x         0
acc_y         0
acc_z         0
activity      0
dtype: int64

In [21]:
# print number of unique subjects
print(f"number of unique subjects: {data['subject_id'].nunique()}")

number of unique subjects: 52


# Windowing Data

In [23]:
def sliding_window_samples(data, samples_per_window, overlap_ratio):
    """
    Return a sliding window measured in number of samples over a data array along with the mode label for each window.

    :param data: input array, can be numpy or pandas dataframe
    :param samples_per_window: window length as number of samples
    :param overlap_ratio: overlap is meant as percentage and should be an integer value
    :return: tuple of windows, indices, and labels
    """
    windows = []
    indices = []
    labels = []
    curr = 0
    win_len = int(samples_per_window)
    if overlap_ratio is not None:
        overlapping_elements = int((overlap_ratio / 100) * win_len)
        if overlapping_elements >= win_len:
            print('Number of overlapping elements exceeds window size.')
            return
    while curr < len(data) - win_len:
        window = data[curr:curr + win_len]
        windows.append(window.iloc[:, 1:-1])  # Exclude the first and last columns
        indices.append([curr, curr + win_len])
        
        # Extract and compute the mode of the encoded labels for the current window
        window_labels = window['subject_id']
        mode_result = mode(window_labels)
        window_label = mode_result[0] if mode_result[0].size > 0 else mode_result
        labels.append(window_label)

        curr += win_len - overlapping_elements

    result_windows = np.array(windows)
    result_indices = np.array(indices)
    result_labels = np.array(labels)
    return result_windows, result_indices, result_labels

In [23]:
sampling_rate = 50
time_window = 2
window_size = sampling_rate * time_window
overlap_ratio = 0

window_data, _, window_subject = sliding_window_samples(data, window_size, overlap_ratio)
print(f"shape of window dataset (2 sec with 0% overlap): {window_data.shape}")

KeyboardInterrupt: 

In [ ]:
window_data[0]

array([[ 0.3065625,  9.196875 , -1.22625  ],
       [ 0.3065625,  9.196875 , -1.22625  ],
       [ 0.3065625,  9.196875 , -1.22625  ],
       [ 0.3065625,  9.196875 , -1.22625  ],
       [ 0.3065625,  9.196875 , -1.22625  ],
       [ 0.3065625,  9.196875 , -1.22625  ],
       [ 0.3065625,  9.196875 , -1.22625  ],
       [ 0.3065625,  9.196875 , -1.22625  ],
       [ 0.3065625,  9.196875 , -1.22625  ],
       [ 0.3065625,  9.196875 , -1.22625  ],
       [ 0.3065625,  9.196875 , -1.22625  ],
       [ 0.3065625,  9.196875 , -1.22625  ],
       [ 0.3065625,  9.196875 , -1.22625  ],
       [ 0.3065625,  9.196875 , -1.22625  ],
       [ 0.3065625,  9.196875 , -1.22625  ],
       [ 0.3065625,  9.196875 , -1.22625  ],
       [ 0.3065625,  9.196875 , -1.22625  ],
       [ 0.3065625,  9.196875 , -1.22625  ],
       [ 0.3065625,  9.196875 , -1.22625  ],
       [ 0.3065625,  9.196875 , -1.22625  ],
       [ 0.3065625,  9.196875 , -1.22625  ],
       [ 0.3065625,  9.196875 , -1.22625  ],
       [ 0

In [ ]:
# # remove the label column
# window_data = window_data[:, :, :-1]
# # window_data = window_data[:, :, :-1]
# #remove the subject column
# window_data = window_data[:, :, 1:]

In [ ]:
window_data[0].shape

(100, 3)

In [ ]:
window_subject.shape

(42864,)

# Signal Transformation

In [24]:
def add_random_noise_single_window(data, noise_level=0.1):
    """
    Add random Gaussian noise to a single window of data.

    :param data: Input data with shape (window_size, 3).
    :param noise_level: Standard deviation of the Gaussian noise.
    :return: Noisy data.
    """
    noise = np.random.normal(0, noise_level, data.shape)
    noisy_data = data + noise
    return noisy_data


In [25]:
def random_cropping_single_window(data, crop_ratio=0.8):
    """
    Randomly crop a single window of data and pad it to maintain original shape.

    :param data: Input data with shape (window_size, 3).
    :param crop_ratio: Ratio of the original window size to keep.
    :return: Cropped and padded data.
    """
    window_size = data.shape[0]
    new_size = int(window_size * crop_ratio)
    start = np.random.randint(0, window_size - new_size)
    end = start + new_size
    cropped_data = data[start:end, :]

    # Pad the cropped data to maintain original window size
    padding_size = window_size - new_size
    padding = np.zeros((padding_size, data.shape[1]))
    padded_data = np.vstack((cropped_data, padding))

    return padded_data


In [26]:
def magnitude_warping_single_window(data, warp_factor=0.2):
    """
    Apply magnitude warping to a single window of data.

    :param data: Input data with shape (window_size, 3).
    :param warp_factor: Factor to determine the magnitude of warping.
    :return: Warped data.
    """
    window_size = data.shape[0]
    warped_data = np.copy(data)

    for j in range(3):  # for each axis
        time_points = np.linspace(0, 1, window_size)
        random_points = np.linspace(0, 1, np.random.randint(4, 10))
        warp_values = 1 + np.random.normal(0, warp_factor, random_points.size)
        interpolator = CubicSpline(random_points, warp_values)
        warped_data[:, j] *= interpolator(time_points)

    return warped_data


In [27]:
def time_warping_single_window(data, warp_factor=0.2):
    """
    Apply time warping to a single window of data.

    :param data: Input data with shape (window_size, 3).
    :param warp_factor: Factor to determine the magnitude of time warping.
    :return: Time-warped data.
    """
    window_size = data.shape[0]
    warped_data = np.zeros_like(data)
    time_points = np.linspace(0, 1, window_size)
    random_points = np.sort(np.random.rand(np.random.randint(3, 6)))
    warp_values = np.random.normal(1, warp_factor, random_points.size)
    interpolator = CubicSpline(random_points, warp_values)
    warped_time = interpolator(time_points)
    warped_time -= warped_time.min()
    warped_time /= warped_time.max()
    warped_time *= (window_size - 1)

    for j in range(3):  # for each axis
        interpolator = CubicSpline(np.arange(window_size), data[:, j])
        warped_data[:, j] = interpolator(warped_time)

    return warped_data


In [28]:
import numpy as np
from scipy.fftpack import fft, ifft

def amplitude_randomization_single_sample(windowed_sample, alpha=1.0):
    """
    Apply Amplitude Randomization to a single windowed sample.
    
    Parameters:
    - windowed_sample: A numpy array of shape (400, 3) representing a single sample.
    - alpha: A parameter controlling the extent of amplitude modulation.
    
    Returns:
    - A numpy array of shape (400, 3) representing the augmented sample.
    """
    # Initialize the output array with the same shape as the input sample
    augmented_sample = np.zeros_like(windowed_sample)

    # Iterate through each channel (x, y, z)
    for channel in range(windowed_sample.shape[1]):
        # Apply Fourier transform to the channel signal
        fft_signal = fft(windowed_sample[:, channel])
        
        # Compute amplitude and phase
        amplitude = np.abs(fft_signal)
        phase = np.angle(fft_signal)
        
        # Generate random amplitude modulation
        random_modulation = np.random.uniform(-alpha, alpha, size=amplitude.shape)
        
        # Modulate the amplitude
        modulated_amplitude = (alpha + random_modulation) * amplitude
        
        # Combine modulated amplitude with the original phase
        fft_augmented = modulated_amplitude * (np.cos(phase) + 1j * np.sin(phase))
        
        # Apply inverse Fourier transform to get the augmented time-series signal
        augmented_signal = ifft(fft_augmented)
        
        # Assign the real part of the reconstructed signal to the output array
        augmented_sample[:, channel] = augmented_signal.real

    return augmented_sample

In [ ]:
# add random noise
noisy_data = add_random_noise_single_window(window_data[0], noise_level=0.1)
print(f"shape of noisy data: {noisy_data.shape}")
# random cropping
cropped_data = random_cropping_single_window(window_data[0], crop_ratio=0.8)
print(f"shape of cropped data: {cropped_data.shape}")
# magnitude warping
warped_data = magnitude_warping_single_window(window_data[0], warp_factor=0.2)
print(f"shape of warped data: {warped_data.shape}")
# time warping
time_warped_data = time_warping_single_window(window_data[0], warp_factor=0.2)
print(f"shape of time warped data: {time_warped_data.shape}")

# amplitude randomization
amplitude_randomized_data = amplitude_randomization_single_sample(window_data[0])
print(f"shape of amplitude randomized data: {amplitude_randomized_data.shape}")

shape of noisy data: (100, 3)
shape of cropped data: (100, 3)
shape of warped data: (100, 3)
shape of time warped data: (100, 3)
shape of amplitude randomized data: (100, 3)


In [ ]:
#make a copy of the original data
augmented_data = np.copy(window_data)

# create labels for augmented data
augmented_labels = []

#create subject id for augmented data
augmented_subject = []


# loop over all windows
for i in range(window_data.shape[0]):
    # choose one number from 0 to 4
    random_number = np.random.randint(0, 4)

    if random_number == 0:
        augmented_data[i] = add_random_noise_single_window(window_data[i], noise_level=0.1)
        augmented_labels.append(0)
        augmented_subject.append(window_subject[i])
    elif random_number == 1:
        augmented_data[i] = random_cropping_single_window(window_data[i], crop_ratio=0.8)
        augmented_labels.append(1)
        augmented_subject.append(window_subject[i])
    elif random_number == 2:
        augmented_data[i] = magnitude_warping_single_window(window_data[i], warp_factor=0.2)
        augmented_labels.append(2)
        augmented_subject.append(window_subject[i])
    elif random_number == 3:
        augmented_data[i] = time_warping_single_window(window_data[i], warp_factor=0.2)
        augmented_labels.append(3)
        augmented_subject.append(window_subject[i])

In [ ]:
#count distribution of labels
unique, counts = np.unique(augmented_labels, return_counts=True)
print(dict(zip(unique, counts)))

{0: 10611, 1: 10923, 2: 10750, 3: 10580}


In [ ]:
# count distribution of subjects
unique, counts = np.unique(augmented_subject, return_counts=True)
print(dict(zip(unique, counts)))

{0: 1290, 1: 1590, 2: 1229, 3: 1792, 4: 1792, 5: 1280, 6: 1280, 7: 1536, 8: 1536, 9: 1024, 10: 1792, 11: 1536, 12: 1536, 13: 1080, 14: 1280, 15: 1380, 16: 1260, 17: 1792, 18: 1280, 19: 1536, 20: 1536, 21: 1280, 22: 399, 23: 346, 24: 384, 25: 336, 26: 319, 27: 486, 28: 332, 29: 319, 30: 319, 31: 372, 32: 330, 33: 328, 34: 350, 35: 340, 36: 344, 37: 422, 38: 408, 39: 425, 40: 369, 41: 377, 42: 403, 43: 354, 44: 381, 45: 401, 46: 416, 47: 410, 48: 372, 49: 394, 50: 365, 51: 426}


In [ ]:
10611 + 10923 + 10750 + 10580

42864

In [ ]:
print(f"shape of augmented data: {augmented_data.shape}")
print(f"shape of augmented labels: {len(augmented_labels)}")
print(f"shape of augmented subject: {len(augmented_subject)}")

shape of augmented data: (42864, 100, 3)
shape of augmented labels: 42864
shape of augmented subject: 42864


In [ ]:
# save augmented data and labels
# np.save('./data_processing/augmented_data_join.npy', augmented_data)
# np.save('./data_processing/augmented_labels_join.npy', augmented_labels)

# np.save('./data_processing/augmented_data_join_2.npy', augmented_data) # 2 datasets
# np.save('./data_processing/augmented_labels_join_2.npy', augmented_labels) # 2 datasets

# np.save('./data_processing/augmented_data_join_3.npy', augmented_data) # 2 datasets, 5 augmentations
# np.save('./data_processing/augmented_labels_join_3.npy', augmented_labels) # 2 datasets, 5 augmentations

# np.save('./data_processing/augmented_data_join_4.npy', augmented_data) # 2 datasets, 4 augmentations, subject id
# np.save('./data_processing/augmented_labels_join_4.npy', augmented_labels) # 2 datasets, 4 augmentations, subject id
# np.save('./data_processing/augmented_subject_join_4.npy', augmented_subject) # 2 datasets, 4 augmentations, subject id

# Model Architecture

## CNN Feature

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ReverseLayerF(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha
        return output, None

class CNNFeatureExtractorWithAdversarial(nn.Module):
    def __init__(self, num_classes=4, num_subjects=52):
        super(CNNFeatureExtractorWithAdversarial, self).__init__()

        # Feature extractor
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(256 * 12, 128)  # Adjust based on your dataset
        
        # Activity classifier
        self.activity_classifier = nn.Linear(128, num_classes)

        # Subject discriminator
        self.subject_discriminator_fc1 = nn.Linear(128, 64)
        self.subject_discriminator_fc2 = nn.Linear(64, num_subjects)

    def forward(self, x, alpha=0.0):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = self.flatten(x)
        features = F.relu(self.fc1(x))

        activity_output = self.activity_classifier(features)

        reversed_features = ReverseLayerF.apply(features, alpha)
        subject_output = F.relu(self.subject_discriminator_fc1(reversed_features))
        subject_output = self.subject_discriminator_fc2(subject_output)

        return activity_output, subject_output


## TCN

In [49]:
class TemporalBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, dilation, padding):
        super(TemporalBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size,
                               stride=stride, padding=0, dilation=dilation)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size,
                               stride=stride, padding=0, dilation=dilation)
        self.relu2 = nn.ReLU()
        self.downsample = nn.Conv1d(in_channels, out_channels, 1) if in_channels != out_channels else None
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.conv2(out)
        out = self.relu2(out)
        
        res = x if self.downsample is None else self.downsample(x)

        # Adjusting the length of the residual to match the output
        if out.size(2) != res.size(2):
            desired_length = out.size(2)
            res = res[:, :, :desired_length]

        return self.relu(out + res)


class TCNWithAdversarial(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size, dropout=0.2, num_classes=4, num_subjects=52):
        super(TCNWithAdversarial, self).__init__()
        # TCN base layers
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size + (dilation_size - 1))]

        self.tcn = nn.Sequential(*layers)
        self.dropout = nn.Dropout(dropout)
        self.fc_activity = nn.Linear(num_channels[-1], num_classes)

        # Subject discriminator
        self.fc_subject = nn.Linear(num_channels[-1], 64)  # Intermediate layer for subject discriminator
        self.subject_discriminator = nn.Linear(64, num_subjects)

    def forward(self, x, alpha=0.0):
        x = self.tcn(x)
        x = F.avg_pool1d(x, x.size(2)).squeeze(2)  # Global Average Pooling
        x = self.dropout(x)

        activity_output = self.fc_activity(x)

        # Adversarial branch for subject discrimination
        reversed_features = ReverseLayerF.apply(x, alpha)
        subject_features = F.relu(self.fc_subject(reversed_features))
        subject_output = self.subject_discriminator(subject_features)

        return activity_output, subject_output

In [31]:
# load augmented data and labels
# augmented_data = np.load('./data_processing/augmented_data_join.npy', allow_pickle=True)
# augmented_labels = np.load('./data_processing/augmented_labels_join.npy', allow_pickle=True)

# augmented_data = np.load('./data_processing/augmented_data_join_2.npy', allow_pickle=True)
# augmented_labels = np.load('./data_processing/augmented_labels_join_2.npy', allow_pickle=True)

# augmented_data = np.load('./data_processing/augmented_data_join_3.npy', allow_pickle=True)
# augmented_labels = np.load('./data_processing/augmented_labels_join_3.npy', allow_pickle=True)

augmented_data = np.load('./data_processing/augmented_data_join_4.npy', allow_pickle=True)
augmented_labels = np.load('./data_processing/augmented_labels_join_4.npy', allow_pickle=True)
augmented_subject = np.load('./data_processing/augmented_subject_join_4.npy', allow_pickle=True)

In [32]:
augmented_data.shape

(42864, 100, 3)

In [33]:
augmented_labels.shape

(42864,)

In [34]:
augmented_subject.shape

(42864,)

In [35]:
#count distribution of labels
unique, counts = np.unique(augmented_labels, return_counts=True)
print(dict(zip(unique, counts)))

{0: 10611, 1: 10923, 2: 10750, 3: 10580}


In [36]:
# count distribution of subjects
unique, counts = np.unique(augmented_subject, return_counts=True)
print(dict(zip(unique, counts)))

{0: 1290, 1: 1590, 2: 1229, 3: 1792, 4: 1792, 5: 1280, 6: 1280, 7: 1536, 8: 1536, 9: 1024, 10: 1792, 11: 1536, 12: 1536, 13: 1080, 14: 1280, 15: 1380, 16: 1260, 17: 1792, 18: 1280, 19: 1536, 20: 1536, 21: 1280, 22: 399, 23: 346, 24: 384, 25: 336, 26: 319, 27: 486, 28: 332, 29: 319, 30: 319, 31: 372, 32: 330, 33: 328, 34: 350, 35: 340, 36: 344, 37: 422, 38: 408, 39: 425, 40: 369, 41: 377, 42: 403, 43: 354, 44: 381, 45: 401, 46: 416, 47: 410, 48: 372, 49: 394, 50: 365, 51: 426}


In [ ]:
# # Convert list of arrays to a single numpy array
# if isinstance(augmented_data, list):
#     augmented_data = np.stack(augmented_data)

# # Ensure the data type is float32
# augmented_data = augmented_data.astype(np.float32)

# # Convert to PyTorch tensors
# augmented_data_tensor = torch.from_numpy(augmented_data)
# augmented_labels_tensor = torch.from_numpy(np.array(augmented_labels)).long()

# # split data into train and test set
# train_size = int(0.8 * augmented_data.shape[0])
# test_size = augmented_data.shape[0] - train_size
# train_data, test_data = torch.utils.data.random_split(augmented_data, [train_size, test_size])
# train_labels, test_labels = torch.utils.data.random_split(augmented_labels, [train_size, test_size])

# # create train and test dataset
# train_dataset = torch.utils.data.TensorDataset(train_data, train_labels)
# test_dataset = torch.utils.data.TensorDataset(test_data, test_labels)

# # create train and test dataloader
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# # check if GPU is available
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# DataLoader Preparation

In [37]:
import numpy as np

# Assuming augmented_data and augmented_labels are numpy arrays
augmented_data = augmented_data.astype(np.float32)
augmented_subject = augmented_subject.astype(np.int64)

# Convert to PyTorch tensors
augmented_data_tensor = torch.from_numpy(augmented_data)
augmented_labels_tensor = torch.from_numpy(augmented_labels)
#convert labels to long
augmented_labels_tensor = augmented_labels_tensor.long()
augmented_subject_tensor = torch.from_numpy(augmented_subject).long()

# Create a dataset with data, labels, and subject IDs
dataset = TensorDataset(augmented_data_tensor, augmented_labels_tensor, augmented_subject_tensor)

# Split data into train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# create DataLoader for train and test sets
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [38]:
#  print the shape of train_loader and test_loader
print(f"shape of train_loader: {len(train_loader)}")
print(f"shape of test_loader: {len(test_loader)}")

shape of train_loader: 268
shape of test_loader: 67


# Train and Test Model

In [43]:
def train_adversarial(model, train_loader, criterion_activity, criterion_subject, optimizer, alpha, device):
    model.train()
    running_loss_activity = 0.0
    running_loss_subject = 0.0
    running_loss = 0.0

    for i, (inputs, labels, subjects) in enumerate(train_loader):
        inputs, labels, subjects = inputs.to(device), labels.to(device), subjects.to(device)
        inputs = inputs.transpose(1, 2)  # Adjust dimensions if necessary

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass to get outputs for both tasks
        activity_output, subject_output = model(inputs, alpha)

        # Calculate separate losses
        loss_activity = criterion_activity(activity_output, labels)
        loss_subject = criterion_subject(subject_output, subjects)

        # Adversarial training: Update model to minimize activity loss and subject loss
        # Note: For adversarial learning, you might adjust the weight of loss_subject or apply gradient reversal within the model
        total_loss = loss_activity - (alpha * loss_subject)  # Subtract subject loss to promote subject-invariant features

        # Backward and optimize
        total_loss.backward()
        optimizer.step()

        # Print statistics or accumulate losses
        running_loss_activity += loss_activity.item()
        running_loss_subject += loss_subject.item()
        running_loss += total_loss.item()

    avg_loss_activity = running_loss_activity / len(train_loader)
    avg_loss_subject = running_loss_subject / len(train_loader)
    avg_loss = running_loss / len(train_loader)

    return avg_loss_activity, avg_loss_subject, avg_loss


In [44]:
# create testing function
def test_adversarial(model, test_loader, criterion_activity, criterion_subject, alpha, device):
    model.eval()
    running_loss_activity = 0.0
    running_loss_subject = 0.0
    #calculate accuracy
    correct = 0
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            inputs, labels, subjects = data[0].to(device), data[1].to(device), data[2].to(device)
            inputs = inputs.transpose(1, 2)
            activity_output, subject_output = model(inputs, alpha)
            loss_activity = criterion_activity(activity_output, labels)
            loss_subject = criterion_subject(subject_output, subjects)

            running_loss_activity += loss_activity.item()
            running_loss_subject += loss_subject.item()

            #calculate accuracy
            _, predicted = torch.max(activity_output.data, 1)
            correct += (predicted == labels).sum().item()
    accuracy = correct / len(test_loader.dataset)
    return running_loss_activity / len(test_loader), running_loss_subject / len(test_loader), accuracy

In [45]:
# create function to train and test model
def train_and_test(model, train_loader, test_loader, criterion_activity, criterion_subject, optimizer, alpha, device, num_epochs):
    train_losses = []
    test_losses = []
    test_accuracies = []
    for epoch in range(num_epochs):
        train_act_loss, train_sub_loss, train_loss = train_adversarial(model, train_loader, criterion_activity, criterion_subject, optimizer, alpha, device)
        test_act_loss, test_sub_los, test_accuracy = test_adversarial(model, test_loader, criterion_activity, criterion_subject, alpha, device)
        train_losses.append(train_loss)
        test_losses.append(test_act_loss)
        test_accuracies.append(test_accuracy)
        print(f"Epoch: {epoch + 1}/{num_epochs}.. Train Loss: {train_loss:.3f}.. Train Act Loss: {train_act_loss:.3f}.. Train Subj Loss: {train_sub_loss:.3f}.. Test Act Loss: {test_act_loss:.3f}.. Test Accuracy: {test_accuracy:.3f}")
    return train_losses, test_losses, test_accuracies

In [47]:
num_classes = 4
num_subjects = 52

model = CNNFeatureExtractorWithAdversarial(num_classes=num_classes, num_subjects=num_subjects)

# move model to GPU if available
model.to(device)

# define loss function and optimizer
criterion_activity = nn.CrossEntropyLoss()
criterion_subject = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
alpha = 0.125

# train and test model
num_epochs = 50
train_losses, test_losses, test_accuracies = train_and_test(model, train_loader, test_loader, criterion_activity, criterion_subject, optimizer, alpha, device, num_epochs)

Epoch: 1/50.. Train Loss: 0.849.. Train Act Loss: 1.344.. Train Subj Loss: 3.963.. Test Act Loss: 1.296.. Test Accuracy: 0.449
Epoch: 2/50.. Train Loss: 0.732.. Train Act Loss: 1.228.. Train Subj Loss: 3.966.. Test Act Loss: 1.146.. Test Accuracy: 0.527
Epoch: 3/50.. Train Loss: 0.579.. Train Act Loss: 1.076.. Train Subj Loss: 3.973.. Test Act Loss: 1.019.. Test Accuracy: 0.537
Epoch: 4/50.. Train Loss: 0.494.. Train Act Loss: 0.992.. Train Subj Loss: 3.984.. Test Act Loss: 0.971.. Test Accuracy: 0.536
Epoch: 5/50.. Train Loss: 0.460.. Train Act Loss: 0.959.. Train Subj Loss: 3.995.. Test Act Loss: 0.949.. Test Accuracy: 0.565
Epoch: 6/50.. Train Loss: 0.441.. Train Act Loss: 0.942.. Train Subj Loss: 4.006.. Test Act Loss: 0.935.. Test Accuracy: 0.533
Epoch: 7/50.. Train Loss: 0.427.. Train Act Loss: 0.929.. Train Subj Loss: 4.018.. Test Act Loss: 0.922.. Test Accuracy: 0.555
Epoch: 8/50.. Train Loss: 0.414.. Train Act Loss: 0.918.. Train Subj Loss: 4.033.. Test Act Loss: 0.913.. Test 

In [33]:
# save model
# timestamp = time.strftime("%Y%m%d-%H%M")
# model_name = f"cnn_feature_extractor_join_2_dataset.pt"
# model_name = f"cnn_feature_extractor_join_2_dataset_5_augmentations.pt"
# torch.save(model.state_dict(), f"./models/{model_name}")

In [52]:
# Example usage
num_inputs = 3  # Assuming 3 input channels (x, y, z axes of the accelerometer)
num_channels = [64, 128, 256]  # Example channel sizes for each layer
kernel_size = 8  # Kernel size for temporal convolutions

model = TCNWithAdversarial(num_inputs, num_channels, kernel_size, num_classes=num_classes, num_subjects=num_subjects).to(device)

# define loss function and optimizer
criterion_activity = nn.CrossEntropyLoss()
criterion_subject = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.00003)  # Replace lr with your learning rate
alpha = 0.125

# train and test model
num_epochs = 85
train_losses, test_losses, test_accuracies = train_and_test(model, train_loader, test_loader, criterion_activity, criterion_subject, optimizer, alpha, device, num_epochs)

In [35]:
#save model
# timestamp = time.strftime("%Y%m%d-%H%M")
# model_name = f"tcn_join_2_dataset.pt"
# model_name = f"tcn_join_2_dataset_5_augmentations.pt"
# torch.save(model.state_dict(), f"./models/{model_name}")